#Import and clean data

##Importing the data

In [ ]:
!pip install -U spacy==3.*
!python -m spacy download en_core_web_sm
!python -m spacy info

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 99.1 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.4
    Uninstalling spacy-3.4.4:
      Successfully uninstalled spacy-3.4.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.4.1 requires spacy<3.5.0,>=3.4.0, but you have spacy 3.5.0 which is incompatible.
2023-02-27 03:02:28.463070: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-27 03:02:29.322387: W tensorflow/compiler/

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
from sklearn import metrics
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [ ]:
training_corpus = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

In [ ]:
target_df = pd.DataFrame(training_corpus.target)
target_df.value_counts, training_corpus.target_names

(<bound method DataFrame.value_counts of         0
 0       7
 1       4
 2       4
 3       1
 4      14
 ...    ..
 11309  13
 11310   4
 11311   3
 11312   1
 11313   8
 
 [11314 rows x 1 columns]>,
 ['alt.atheism',
  'comp.graphics',
  'comp.os.ms-windows.misc',
  'comp.sys.ibm.pc.hardware',
  'comp.sys.mac.hardware',
  'comp.windows.x',
  'misc.forsale',
  'rec.autos',
  'rec.motorcycles',
  'rec.sport.baseball',
  'rec.sport.hockey',
  'sci.crypt',
  'sci.electronics',
  'sci.med',
  'sci.space',
  'soc.religion.christian',
  'talk.politics.guns',
  'talk.politics.mideast',
  'talk.politics.misc',
  'talk.religion.misc'])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(training_corpus.data, training_corpus.target, train_size=0.8, random_state=1) 

In [ ]:
#On 10000 rows, count vectorizer gives f1 of 0.15
#On 10000 rows, count vectorizer gives f1 of 0.16
#One reason for above small f1 might be target data is imbalanced
#After removing target 5 which creates bias, F1 score on initial testing set: 0.46145917730330815 
# F1 score on initial testing set on Random Forest: 0.5051871671334535
# F1 score on initial testing set on Random Forest: 0.5051871671334535
# F1 score on initial testing set on Decision Tree: 0.46145917730330815


##Feature Processing

-> Remove all the characters

-> Keep or remove stop words? We'll see

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [ ]:
unwanted_pipes = ['ner', 'parser']

# Further remove stop words and take the lemma instead of token text.
def spacy_tokenizer(doc):
  with nlp.disable_pipes(*unwanted_pipes):
    return [t.lemma_ for t in nlp(doc) if \
            ##Try with and without punctuation, as punctuation holds some sentiment/emotion value
            not t.is_punct and \
            not t.is_space and \
            not t.is_stop and \
            t.is_alpha]

In [ ]:
%%time
vectorizer = TfidfVectorizer(tokenizer=spacy_tokenizer)
train_feature_vects = vectorizer.fit_transform(x_train)

CPU times: user 2min 46s, sys: 4.38 s, total: 2min 51s
Wall time: 2min 55s


##Decision Trees

In [ ]:
from sklearn.tree import DecisionTreeClassifier
# Instantiate a classifier with the default settings.
nb_classifier = DecisionTreeClassifier()
nb_classifier.fit(train_feature_vects, y_train)
nb_classifier.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': None,
 'splitter': 'best'}

In [ ]:
train_preds = nb_classifier.predict(train_feature_vects)
print('F1 score on initial training set: {}'.format(metrics.f1_score(y_train, train_preds, average='macro')))

F1 score on initial training set: 0.9761751772180869


In [ ]:
test_feature_vects = vectorizer.transform(x_test)

In [ ]:
test_preds = nb_classifier.predict(test_feature_vects)
print('F1 score on initial testing set: {}'.format(metrics.f1_score(y_test, test_preds, average='macro')))

F1 score on initial testing set: 0.46273103763957135


##Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

nb_classifier = RandomForestClassifier()
nb_classifier.fit(train_feature_vects, y_train)
nb_classifier.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [ ]:
train_preds = nb_classifier.predict(train_feature_vects)
print('F1 score on initial training set: {}'.format(metrics.f1_score(y_train, train_preds, average='macro')))

F1 score on initial training set: 0.9761768088155037


In [ ]:
test_preds = nb_classifier.predict(test_feature_vects)
print('F1 score on initial testing set: {}'.format(metrics.f1_score(y_test, test_preds, average='macro')))

F1 score on initial testing set: 0.6332794689414426
